In [16]:
import pandas as pd

In [17]:
df = pd.read_csv("accident data.csv")

In [18]:
df.head(10)

,Index,Accident_Severity,Accident Date,Latitude,Light_Conditions,District Area,Longitude,Number_of_Casualties,Number_of_Vehicles,Road_Surface_Conditions,Road_Type,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type
0,200701BS64157,Serious,05-06-2019,51.506187,Darkness - lights lit,Kensington and Chelsea,-0.209082,1,2,Dry,Single carriageway,Urban,Fine no high winds,Car
1,200701BS65737,Serious,02-07-2019,51.495029,Daylight,Kensington and Chelsea,-0.173647,1,2,Wet or damp,Single carriageway,Urban,Raining no high winds,Car
2,200701BS66127,Serious,26-08-2019,51.517715,Darkness - lighting unknown,Kensington and Chelsea,-0.210215,1,3,Dry,NaN,Urban,NaN,Taxi/Private hire car
3,200701BS66128,Serious,16-08-2019,51.495478,Daylight,Kensington and Chelsea,-0.202731,1,4,Dry,Single carriageway,Urban,Fine no high winds,Bus or coach (17 or more pass seats)
4,200701BS66837,Slight,03-09-2019,51.488576,Darkness - lights lit,Kensington and Chelsea,-0.192487,1,2,Dry,NaN,Urban,NaN,Other vehicle
5,200701BS67159,Serious,18-09-2019,51.497750,Daylight,Kensington and Chelsea,-0.192556,2,3,Dry,Single carriageway,Urban,Fine no high winds,Car
6,200701BS67207,Serious,05-09-2019,51.501405,Daylight,Kensington and Chelsea,-0.161577,1,2,Dry,Dual carriageway,Urban,Fine no high winds,Van / Goods 3.5 tonnes mgw or under
7,200701BS67370,Fatal,03-10-2019,51.482260,Darkness - lights lit,Kensington and Chelsea,-0.179486,3,2,Dry,Single carriageway,Urban,Fine no high winds,Car
8,200701BS67515,Slight,31-10-2019,51.493319,Darkness - lights lit,Kensington and Chelsea,-0.173572,1,2,Dry,Single carriageway,Urban,Fine no high winds,Car
9,200701BS67543,Slight,18-10-2019,51.484539,Daylight,Kensington and Chelsea,-0.175651,1,2,Dry,Single carriageway,Urban,Fine no high winds,Motorcycle over 125cc and up to 500cc


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660679 entries, 0 to 660678
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Index                    660679 non-null  object 
 1   Accident_Severity        660679 non-null  object 
 2   Accident Date            660679 non-null  object 
 3   Latitude                 660654 non-null  float64
 4   Light_Conditions         660679 non-null  object 
 5   District Area            660679 non-null  object 
 6   Longitude                660653 non-null  float64
 7   Number_of_Casualties     660679 non-null  int64  
 8   Number_of_Vehicles       660679 non-null  int64  
 9   Road_Surface_Conditions  659953 non-null  object 
 10  Road_Type                656159 non-null  object 
 11  Urban_or_Rural_Area      660664 non-null  object 
 12  Weather_Conditions       646551 non-null  object 
 13  Vehicle_Type             660679 non-null  object 
dtypes: f

In [20]:
df.isna().sum()

Index                          0
Accident_Severity              0
Accident Date                  0
Latitude                      25
Light_Conditions               0
District Area                  0
Longitude                     26
Number_of_Casualties           0
Number_of_Vehicles             0
Road_Surface_Conditions      726
Road_Type                   4520
Urban_or_Rural_Area           15
Weather_Conditions         14128
Vehicle_Type                   0
dtype: int64

## Data Cleaning

**Convert Data column to datetime**

In [21]:
df['Accident Date'] = df['Accident Date'].str.replace('-', '/', regex=True)
df['Accident Date'] = pd.to_datetime(df['Accident Date'], format='%d/%m/%Y', errors='coerce')

In [22]:
df = df.dropna()

In [23]:
df = df.drop_duplicates()

In [24]:
df["Weather_Conditions"] = df["Weather_Conditions"].str.title().str.strip().str.replace(r'\s*\+\s*', ' & ', regex=True)
df["Road_Type"] = df["Road_Type"].str.title().str.strip()
df["Light_Conditions"] = df["Light_Conditions"].str.title().str.strip()

In [25]:
road_surface_mapping = {
    "Wet or damp": "Wet",
    "Frost or ice": "Icy",
    "Flood over 3cm. deep": "Flooded"
}

# Apply the mapping
df["Road_Surface_Conditions"] = df["Road_Surface_Conditions"].replace(road_surface_mapping)


In [26]:
district_mapping = {
    "Clackmannshire": "Clackmannanshire",
    "Crewe and ntwich": "Crewe and Nantwich",
    "Blaeu Gwent": "Blaenau Gwent",
    "Stevege": "Stevenage",
    "North Larkshire": "North Lanarkshire",
    "Kingston upon Hull, City of": "Kingston upon Hull",
    "Bristol, City of": "Bristol",
    "Edinburgh, City of": "Edinburgh",
    "Rhondda, Cynon, Taff": "Rhondda Cynon Taff",
}

df["District Area"] = df["District Area"].replace(district_mapping)
df["District Area"] = df["District Area"].str.title()

In [27]:
vehicle_mapping = {
    # Merge all motorcycles into one category
    "Motorcycle 50cc and under": "Motorcycle",
    "Motorcycle 125cc and under": "Motorcycle",
    "Motorcycle over 125cc and up to 500cc": "Motorcycle",
    "Motorcycle over 500cc": "Motorcycle",
    
    # Merge heavy goods vehicles (HGVs)
    "Goods over 3.5t. and under 7.5t": "Heavy Goods Vehicle (HGV)",
    "Goods 7.5 tonnes mgw and over": "Heavy Goods Vehicle (HGV)",
    
    # Standardize light goods vehicle (LGV)
    "Van / Goods 3.5 tonnes mgw or under": "Light Goods Vehicle (LGV)",
    
    # Rename "Data missing or out of range" to "Other Vehicle"
    "Data missing or out of range": "Other Vehicle"
}

# Apply the mapping
df["Vehicle_Type"] = df["Vehicle_Type"].replace(vehicle_mapping)
df["Vehicle_Type"] = df["Vehicle_Type"].str.title()

In [28]:
df

,Index,Accident_Severity,Accident Date,Latitude,Light_Conditions,District Area,Longitude,Number_of_Casualties,Number_of_Vehicles,Road_Surface_Conditions,Road_Type,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type
0,200701BS64157,Serious,2019-06-05,51.506187,Darkness - Lights Lit,Kensington And Chelsea,-0.209082,1,2,Dry,Single Carriageway,Urban,Fine No High Winds,Car
1,200701BS65737,Serious,2019-07-02,51.495029,Daylight,Kensington And Chelsea,-0.173647,1,2,Wet,Single Carriageway,Urban,Raining No High Winds,Car
3,200701BS66128,Serious,2019-08-16,51.495478,Daylight,Kensington And Chelsea,-0.202731,1,4,Dry,Single Carriageway,Urban,Fine No High Winds,Bus Or Coach (17 Or More Pass Seats)
5,200701BS67159,Serious,2019-09-18,51.497750,Daylight,Kensington And Chelsea,-0.192556,2,3,Dry,Single Carriageway,Urban,Fine No High Winds,Car
6,200701BS67207,Serious,2019-09-05,51.501405,Daylight,Kensington And Chelsea,-0.161577,1,2,Dry,Dual Carriageway,Urban,Fine No High Winds,Light Goods Vehicle (Lgv)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660674,201091NM01760,Slight,2022-02-18,57.374005,Daylight,Highland,-3.467828,2,1,Dry,Single Carriageway,Rural,Fine No High Winds,Car
660675,201091NM01881,Slight,2022-02-21,57.232273,Darkness - No Lighting,Highland,-3.809281,1,1,Icy,Single Carriageway,Rural,Fine No High Winds,Car
660676,201091NM01935,Slight,2022-02-23,57.585044,Daylight,Highland,-3.862727,1,3,Icy,Single Carriageway,Rural,Fine No High Winds,Car
660677,201091NM01964,Serious,2022-02-23,57.214898,Darkness - No Lighting,Highland,-3.823997,1,2,Wet,Single Carriageway,Rural,Fine No High Winds,Motorcycle


In [29]:
df['Vehicle_Type'].unique()

array(['Car', 'Bus Or Coach (17 Or More Pass Seats)',
       'Light Goods Vehicle (Lgv)', 'Motorcycle',
       'Heavy Goods Vehicle (Hgv)', 'Taxi/Private Hire Car',
       'Other Vehicle', 'Minibus (8 - 16 Passenger Seats)', 'Pedal Cycle',
       'Agricultural Vehicle', 'Ridden Horse'], dtype=object)

In [30]:
df.to_csv('cleaned_accidents.csv', index=False)